In [ ]:
import ee 

class NDVI:
    def __init__(self,region,start_date,end_date,name ="Unnamed"):
        self.region = region
        self.start_date = start_date
        self.end_date = end_date
        self.name = name
        
        self.collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
        self.Image = ee.Image(self.collection.filterDate(self.start_date,self.end_date).filterBounds(self.region).sort('CLOUD_COVER').map(self._add_ndvi))
    
    def _add_ndvi(self,img):
        ndvi = img.normalizedDifference(['B5', 'B4']).rename('NDVI')
        return ndvi.copyProperties(img, ['system:time_start'])
    
    def get_timeseries(self):
        def reduce_img(img):
            stat = img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=self.region.geometry(),
                scale=250,
                maxPixels=1e13
            )
            return ee.Feature(None, {
                "NDVI": stat.get("NDVI"),
                "date": img.date().format("YYYY-MM-dd"),
                "name": self.name
            })
        return self.collection.map(reduce_img)


In [ ]:
import ee
import folium
from datetime import datetime
from IPython.display import Image

ee.Authenticate()
ee.Initialize(project='sgrd-smart-farming')
region = ee.FeatureCollection("projects/sgrd-smart-farming/assets/Areas/SECTOR_1_SUB_1_NORTH_WEST")
print(region.getInfo())
# --------- Load Landsat RGB Image ---------
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterDate('2023-01-01', '2023-12-31') \
    .filterBounds(region) \
    .sort('CLOUD_COVER') \
    .median() \
    .clip(region)

rgb_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.05,
    'max': 0.4
}

rgb_mapid = landsat.getMapId(rgb_vis)

# --------- Compute NDVI Layer ---------
ndvi = landsat.normalizedDifference(['B5', 'B4']).rename('NDVI').clip(region)

ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
]
}

ndvi_mapid = ndvi.getMapId(ndvi_vis)

# --------- Create Folium Map ---------
m = folium.Map(location=[lat, lon], zoom_start=11)

# Add Landsat RGB Base
folium.TileLayer(
    tiles=rgb_mapid['tile_fetcher'].url_format,
    attr='Landsat 8 RGB',
    name='Satellite View',
    overlay=False,
    control=True
).add_to(m)

# Add NDVI Overlay
folium.TileLayer(
    tiles=ndvi_mapid['tile_fetcher'].url_format,
    attr='NDVI Layer',
    name='NDVI',
    overlay=True,
    control=True
).add_to(m)

# Add AOI boundary
folium.GeoJson(region.getInfo(), name='AOI').add_to(m)

# Layer control
folium.LayerControl().add_to(m)

m
# Uncomment to run the NDVI class
#ndvi_obj = NDVI(region, "2013-01-01", "2024-01-01", "SECTOR_1_SUB_1_NORTH_WEST")
#fc = ndvi_obj.get_timeseries()
# Convert to local lists for plotting
#print(fc.getInfo())
#dates = fc.aggregate_array("date").getInfo()
#values = fc.aggregate_array("NDVI").getInfo()

{'type': 'FeatureCollection', 'columns': {'level': 'String', 'name': 'String', 'sector': 'String', 'system:index': 'String'}, 'version': 1745218214420254, 'id': 'projects/sgrd-smart-farming/assets/Areas/SECTOR_1_SUB_1_NORTH_WEST', 'properties': {'system:asset_size': 3251}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[38.28309390198947, 36.885416212771545], [38.08396668310257, 36.86014863188809], [37.99332950173721, 36.80738929950559], [38.061994058152976, 36.66211370254], [38.18559022678317, 36.633466465584874], [38.224042389741804, 36.5077333778019], [38.456128530895185, 36.49007032012907], [38.527539650124545, 36.61362752931979], [38.483594391029804, 36.69295271456372], [38.483594391029804, 36.78099601610058], [38.39021054331311, 36.85135798971317], [38.28309390198947, 36.885416212771545]]]}, 'id': '00000000000000000000', 'properties': {'level': 'subsector', 'name': 'SECTOR_1_SUB_1_NORTH_WEST', 'sector': 'SECTOR_1'}}]}


In [50]:
import ee
import folium
from datetime import datetime
from IPython.display import Image

def scaling(img):
    scaled = img.select('NDVI').divide(10000)
    return scaled.copyProperties(img, ['system:index' , 'system:time_start'])

ee.Authenticate()
ee.Initialize(project='sgrd-smart-farming')
region = ee.FeatureCollection("projects/sgrd-smart-farming/assets/Areas/SECTOR_1_SUB_1_NORTH_WEST")
print(region.getInfo())

dataset = ee.ImageCollection('MODIS/061/MOD13Q1').filter(ee.Filter.date('2014-01-01', '2014-12-31')).filterBounds(region).map(scaling)
print(dataset.size().getInfo())
ndvi = dataset.select('NDVI').mean().clip(region)


ndvi_palette = [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
]

ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': ndvi_palette
}
ndvi_mapid = ndvi.getMapId(ndvi_vis)

# 3. Create folium map
m = folium.Map(location=[lat, lon], zoom_start=11)

# Add Landsat RGB
folium.TileLayer(
    tiles=rgb_mapid['tile_fetcher'].url_format,
    attr='Landsat RGB',
    name='Landsat RGB',
    overlay=False
).add_to(m)

# Add NDVI overlay
folium.TileLayer(
    tiles=ndvi_mapid['tile_fetcher'].url_format,
    attr='NDVI',
    name='NDVI',
    overlay=True
).add_to(m)

# Add AOI boundary
folium.GeoJson(region.getInfo(), name='AOI').add_to(m)

# Layer toggle
folium.LayerControl().add_to(m)

# Display map
m


{'type': 'FeatureCollection', 'columns': {'level': 'String', 'name': 'String', 'sector': 'String', 'system:index': 'String'}, 'version': 1745218214420254, 'id': 'projects/sgrd-smart-farming/assets/Areas/SECTOR_1_SUB_1_NORTH_WEST', 'properties': {'system:asset_size': 3251}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[38.28309390198947, 36.885416212771545], [38.08396668310257, 36.86014863188809], [37.99332950173721, 36.80738929950559], [38.061994058152976, 36.66211370254], [38.18559022678317, 36.633466465584874], [38.224042389741804, 36.5077333778019], [38.456128530895185, 36.49007032012907], [38.527539650124545, 36.61362752931979], [38.483594391029804, 36.69295271456372], [38.483594391029804, 36.78099601610058], [38.39021054331311, 36.85135798971317], [38.28309390198947, 36.885416212771545]]]}, 'id': '00000000000000000000', 'properties': {'level': 'subsector', 'name': 'SECTOR_1_SUB_1_NORTH_WEST', 'sector': 'SECTOR_1'}}]}
23
